In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import os
import random
from collections import defaultdict

## Generate User_Ratings

In [2]:
data_path = './data/saved/user_item_filter_t1000.json'
df = pd.read_json(data_path)

In [ ]:
def generate_user_ratings(df):
    '''
    As for bpr experiment, all ratings are removed.
    '''
    user_ratings = defaultdict(set)
    df_t = df.T
    df_t = df_t.loc[:, (df_t != 0).any(axis=0)]
    for column in df_t:
        games_owned = set(df_t.loc[df_t[column] == 1].index)
        user_ratings[column] = games_owned
    user_count = df_t.shape[1]
    item_count = df_t.shape[0]
    return user_ratings

user_ratings = generate_user_ratings(df)

with open('./data/saved/user_ratings.pkl', 'wb') as file:
    pickle.dump(user_ratings,file)

## Reload Data

In [3]:
user_ratings = pickle.load(open('./data/saved/user_ratings.pkl','rb'))

## Data Preprocessing

In [98]:
def generate_counts(df):
    user_count = df.shape[0]
    item_count = df.shape[1]
    return user_count, item_count

user_count, item_count = generate_counts(df)

In [99]:
def generate_test(user_ratings):
    '''
    for each user, random select one of his(her) rating into test set
    '''
    user_test = dict()
    for u, i_list in user_ratings.items():
        if len(user_ratings[u]) > 1:
            user_test[u] = random.sample(user_ratings[u], 1)[0]
    return user_test

user_ratings_test = generate_test(user_ratings)

In [100]:
def list_games(df):
    games = set(df.columns)
    return games

all_games = list_games(df)

In [101]:
def create_user_item_indices(df):
    user_dict = {}
    i = 0
    for u in df.index:
        user_dict[u] = i
        i += 1
    item_dict = {}
    j = 0
    for g in df.columns:
        item_dict[g] = j
        j += 1
    return user_dict, item_dict

user_dict, item_dict = create_user_item_indices(df)

In [102]:
def generate_train_batch(user_ratings, user_ratings_test, batch_size=512):
    '''
    uniform sampling (user, item_rated, item_not_rated)
    '''
    t = []
    for b in range(batch_size):
        u = random.sample(user_ratings.keys(), 1)[0]
        i = random.sample(user_ratings[u], 1)[0]
        if len(user_ratings[u]) == 1:
            continue
        else:
            while i == user_ratings_test[u]:
                i = random.sample(user_ratings[u], 1)[0]
        j = random.sample(all_games, 1)[0]
        while j in user_ratings[u]:
            j = random.sample(all_games, 1)[0]
        t.append([user_dict[u], item_dict[i], item_dict[j]])
    return np.asarray(t)

def generate_test_batch(user_ratings, user_ratings_test):
    '''
    for an user u and an item i rated by u, 
    generate pairs (u,i,j) for all item j which u has't rated
    it's convinent for computing AUC score for u
    '''
    for u in user_ratings_test.keys():
        t = []
        i = user_ratings_test[u]
        for j in all_games:
            if not (j in user_ratings[u]):
                t.append([user_dict[u], item_dict[i], item_dict[j]])
        yield np.asarray(t)

In [103]:
uij = generate_train_batch(user_ratings, user_ratings_test)

## BPR model

In [104]:
def bpr_mf(user_count, item_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [user_count, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [item_count, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        item_b = tf.get_variable("item_b", [item_count+1, 1], 
                                initializer=tf.constant_initializer(0.0))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        i_b = tf.nn.embedding_lookup(item_b, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
        j_b = tf.nn.embedding_lookup(item_b, j)
    
    # MF predict: u_i > u_j
    x = i_b - j_b + tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keepdims=True)
    
    # AUC for one user:
    # reasonable iff all (u,i,j) pairs are from the same user
    # 
    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.cast(x > 0, tf.float32))
    
    l2_norm = tf.add_n([
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

## Train and Test

In [105]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(user_count, item_count, 20)
    session.run(tf.global_variables_initializer())
    for epoch in range(1, 11):
        _batch_bprloss = 0
        for k in range(1, 1000): # uniform samples from training set
            uij = generate_train_batch(user_ratings, user_ratings_test, 2048)

            _bprloss, _ = session.run([bprloss, train_op], 
                                feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        print ("epoch: ", epoch)
        print ("bpr_loss: ", _batch_bprloss / k)

        user_count = 0
        _auc_sum = 0.0

        # each batch will return only one user's auc
        for t_uij in generate_test_batch(user_ratings, user_ratings_test):

            _auc, _test_bprloss = session.run([mf_auc, bprloss],
                                    feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]}
                                )
            user_count += 1
            _auc_sum += _auc
        print ("test_loss: ", _test_bprloss, "test_auc: ", _auc_sum/user_count)
        print ("")

epoch:  1
bpr_loss:  0.7465756655455351
test_loss:  0.7145543 test_auc:  0.6137979265968185

epoch:  2
bpr_loss:  0.7350447830017861
test_loss:  0.7104371 test_auc:  0.6614741690194178

epoch:  3
bpr_loss:  0.7243671677969359
test_loss:  0.70648265 test_auc:  0.6912428616941262

epoch:  4
bpr_loss:  0.714773994845313
test_loss:  0.70251167 test_auc:  0.7125558887894933

epoch:  5
bpr_loss:  0.7055912030351771
test_loss:  0.6984614 test_auc:  0.7286598914160086

epoch:  6
bpr_loss:  0.6971663386375457
test_loss:  0.6946807 test_auc:  0.7412474658486381

epoch:  7
bpr_loss:  0.6893177674458669
test_loss:  0.6910745 test_auc:  0.7512846780581909

epoch:  8
bpr_loss:  0.6820563679939514
test_loss:  0.6872815 test_auc:  0.7594219256136402

epoch:  9
bpr_loss:  0.6753572800734619
test_loss:  0.6836307 test_auc:  0.7660934601053838

epoch:  10
bpr_loss:  0.6689126338806
test_loss:  0.6799412 test_auc:  0.7716959252266119

